In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import datetime
import pandas as pd
import numpy as np

In [4]:
all_data = pd.read_csv("data.csv")
all_data.head() 

,Row,Participant,Phase,Day,Date,MET_Session,Exercise_Duration_Total,Exercise_METmin_Total,%Complete_total,Mean_total,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,1,1,1,1,12/2/19,0,0,0.0,99.30555556,114.472028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,12/3/19,0,0,0.0,100,132.28125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,3,12/4/19,0,0,0.0,100,141.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1,4,12/5/19,0,0,0.0,82.63888889,108.0840336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,1,5,12/6/19,0,0,0.0,97.56944444,126.13879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = all_data[["Participant","Date","Phase","%Complete_total","Exercise_Duration_Total", "Exercise_METmin_Total", "Planned Day Off_binary", "Not Enough Time_binary", 
               "Lacked Positive Feelings_binary", "Had Negative Feelings_binary", "Lacked Energy_binary", 
              "Felt Fatigued_binary", "Blood sugar management issue_binary", "Sick", "Morning Fear of Hypoglycemia",
              "Evening Fear of Hypoglycemia", "Sleep Quality", "FearOfHypoglycemia", "Felt Fatigued", "Had Negative Feelings"]]
df

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,Lacked Energy_binary,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings
0,1,12/2/19,1,99.30555556,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
1,1,12/3/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
2,1,12/4/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
3,1,12/5/19,1,82.63888889,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
4,1,12/6/19,1,97.56944444,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,23,11/7/20,3,99.30555556,60,120.0,N/.A,N/.A,N/.A,N/.A,N/.A,N/.A,N/.A,0,0,1,5,N/.A,NaN,NaN
1961,23,11/8/20,3,45.83333333,0,0.0,0,0,0,0,1,1,0,0,3,0,5,99,1,99
1962,23,11/9/20,3,82.98611111,0,0.0,0,1,0,0,0,0,0,0,0,0,4,99,99,99
1963,23,11/10/20,3,100,0,0.0,0,1,0,0,0,0,0,0,0,0,5,99,99,99


In [6]:
# Read and combine data
def read_and_combine_librelink_without_parts(participantNumber, high=400, low=40, key="Historic Glucose mg/dL"):
    patientGlucoseDf = pd.read_csv("glucose_data/patient_"+str(participantNumber)+".csv")
    patientGlucoseDf = patientGlucoseDf.dropna(subset=[key])
    patientGlucoseDf[key] = patientGlucoseDf[key].astype(float)
    return patientGlucoseDf

def read_and_combine_dexcom_without_parts(participantNumber, high=400, low=40, key="Glucose Value (mg/dL)"):
    patientGlucoseDf = pd.read_csv("glucose_data/patient_"+str(participantNumber)+".csv")
    patientGlucoseDf = patientGlucoseDf.dropna(subset=[key])
#     patientGlucoseDf[key].replace({"High": high, "Low": low}, inplace=True)
    patientGlucoseDf[key] = patientGlucoseDf[key].astype(float)
    return patientGlucoseDf

def read_and_combine_dexcom_with_parts(participantNumber, high=400, low=40, key="Glucose Value (mg/dL)"):
    patientGlucoseDf_1 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_1.csv")
    patientGlucoseDf_2 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_2.csv")
    patientGlucoseDf = pd.concat([patientGlucoseDf_1, patientGlucoseDf_2])
    patientGlucoseDf = patientGlucoseDf.dropna(subset=[key])
    patientGlucoseDf[key].replace({"High": high, "Low": low}, inplace=True)
    patientGlucoseDf[key] = patientGlucoseDf[key].astype(float)
    return patientGlucoseDf

# Patient One Read Data and Combine
def read_and_combine_carelink_with_parts(participantNumber):
    patientGlucoseDf_1 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_1.csv")
    patientGlucoseDf_2 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_2.csv")
#     patientGlucoseDf_1 = patientGlucoseDf_1.drop(['patient_1_1'], axis=1)
#     patientGlucoseDf_2 = patientGlucoseDf_2.drop(['Index'], axis=1)
    patientGlucoseDf = pd.concat([patientGlucoseDf_1, patientGlucoseDf_2])
    return patientGlucoseDf.dropna(subset=['Sensor Glucose (mg/dL)'])

# fix time and sort by time
def fix_time(patientGlucoseDf):
    dates = []
    for timestamp in patientGlucoseDf["Timestamp (YYYY-MM-DDThh:mm:ss)"]:
        d = datetime.datetime.strptime(timestamp[0:10], '%Y-%m-%d')
        just_day = datetime.datetime.strftime(d, '%d').lstrip('0')
        just_month = datetime.datetime.strftime(d, '%m').lstrip('0')
        just_year = datetime.datetime.strftime(d, '%y')
        dates.append(just_month+'/'+just_day+'/'+just_year)
    patientGlucoseDf["Date"] = dates
    return patientGlucoseDf

def fix_time_librelink(patientGlucoseDf):
    dates = []
    for timestamp in patientGlucoseDf["Device Timestamp"]:
        dates.append(timestamp[0:9])
    patientGlucoseDf["Date"] = dates
    return patientGlucoseDf

# Combine Data on a day level
def agreg_on_day_level(key, patientGlucoseDf):
    
    patientGlucoseDf = patientGlucoseDf[['Date', key]]
    aggregation_functions = {'Date': 'first', key: 'mean'}
    return patientGlucoseDf.groupby(patientGlucoseDf['Date']).aggregate(aggregation_functions)

# Search for corresponding data and return patient dataframe with glucose as column
def search_for_corresponding_data(participantNumber, patientGlucoseDf, key):
    df_patient = df.loc[df['Participant'] == participantNumber]
    glucose_values = []
    for date in list(df_patient['Date']):
        temp = patientGlucoseDf.loc[patientGlucoseDf['Date'] == date]
        if temp.shape[0] == 1:
            val = temp[key].values[0]
            glucose_values.append(val)
        else:    
            glucose_values.append(np.nan)
    df_patient["Sensor Glucose (mg/dL)"] = glucose_values
    df_patient = df_patient.loc[df_patient['%Complete_total'].astype(float) > 70]	
    return df_patient

In [6]:
# Carelink Data PreProcessing
patient_1 = read_and_combine_carelink_with_parts(1)
patient_1 = agreg_on_day_level('Sensor Glucose (mg/dL)', patient_1)
df_patient_1 = search_for_corresponding_data(1, patient_1, 'Sensor Glucose (mg/dL)')
df_patient_1

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
0,1,12/2/19,1,99.30555556,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),115.125874
1,1,12/3/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),134.065972
2,1,12/4/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),130.659722
3,1,12/5/19,1,82.63888889,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),126.756944
4,1,12/6/19,1,97.56944444,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),118.207792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1,3/17/20,3,97.2,0,0.0,0,1,0,0,...,0,0,0,.,0,9,99,99,99,100.367347
93,1,3/18/20,3,96.5,0,0.0,0,0,0,1,...,0,0,0,0,0,5,99,99,1,152.575540
94,1,3/19/20,3,94.8,0,0.0,0,0,0,0,...,0,0,0,0,0,7,99,99,99,141.222222
95,1,3/20/20,3,92.0,0,0.0,.,.,.,.,...,.,.,.,.,.,.,.,.,.,125.584000


In [7]:
# Dexcom Data Preprocessing
patient_2 = read_and_combine_dexcom_with_parts(2)
patient_2 = fix_time(patient_2)
patient_2 = agreg_on_day_level('Glucose Value (mg/dL)', patient_2)
df_patient_2 = search_for_corresponding_data(2, patient_2, 'Glucose Value (mg/dL)')
df_patient_2

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
98,2,12/3/19,1,93.75,30,315.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),155.024306
99,2,12/4/19,1,70.48611111,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),148.263941
100,2,12/5/19,1,81.25,30,315.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),113.700000
101,2,12/6/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),126.982639
102,2,12/7/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),108.881944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,2,3/11/20,3,96.52777778,0,0.0,0,1,0,0,...,1,0,0,0,0,9,99,2,99,189.118056
192,2,3/12/20,3,100,0,0.0,.,.,.,.,...,.,.,.,1,.,.,.,.,.,218.253472
193,2,3/13/20,3,100,0,0.0,0,0,0,0,...,0,0,0,.,0,8,99,99,99,184.350694
194,2,3/14/20,3,100,60,420.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,7,N/.A,NaN,NaN,183.927083


In [8]:
# Carelink Data PreProcessing
patient_4 = read_and_combine_carelink_with_parts(4)
patient_4 = agreg_on_day_level('Sensor Glucose (mg/dL)', patient_4)
df_patient_4 = search_for_corresponding_data(4, patient_1, 'Sensor Glucose (mg/dL)')
df_patient_4

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
196,4,12/5/19,1,99.65277778,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),126.756944
197,4,12/6/19,1,96.875,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),118.207792
200,4,12/9/19,1,92.36111111,60,300.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),144.081272
201,4,12/10/19,1,99.65277778,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),133.729167
202,4,12/11/19,1,100,60,300.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),116.284173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,4,3/18/20,3,99.65277778,0,0.0,0,0,0,0,...,0,0,2,0,0,7,99,99,99,152.575540
290,4,3/19/20,3,98.95833333,0,0.0,0,0,0,0,...,0,0,2,0,0,8,99,99,99,141.222222
291,4,3/20/20,3,99.30555556,0,0.0,0,0,0,0,...,0,0,2,0,0,6,99,99,99,125.584000
292,4,3/21/20,3,99.65277778,0,0.0,0,0,0,0,...,0,0,2,0,0,7,99,99,99,153.527559


In [9]:
# Carelink Data PreProcessing
patient_5 = read_and_combine_carelink_with_parts(5)
patient_5 = agreg_on_day_level('Sensor Glucose (mg/dL)', patient_5)
df_patient_5 = search_for_corresponding_data(5, patient_5, 'Sensor Glucose (mg/dL)')
df_patient_5

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
294,5,12/6/19,1,72.22222222,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),154.810185
295,5,12/7/19,1,82.29166667,90,585.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),151.745283
304,5,12/16/19,1,75,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),176.917391
305,5,12/17/19,1,80.20833333,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),151.993750
306,5,12/18/19,1,93.05555556,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),203.705224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,5,3/25/20,3,99.0,0,0.0,0,0,0,0,...,0,0,0,0,0,6,99,99,99,155.671329
388,5,3/26/20,3,97.2,0,0.0,.,.,.,.,...,.,.,0,0,1,6,.,.,.,130.926316
389,5,3/27/20,3,99.7,31,279.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,7,N/.A,NaN,NaN,153.250000
390,5,3/28/20,3,84.7,0,0.0,1,0,0,0,...,0,0,0,0,0,6,99,99,99,139.664032


In [10]:
# dexcom
patient_6 = read_and_combine_dexcom_with_parts(6)
patient_6 = fix_time(patient_6)
patient_6 = agreg_on_day_level('Glucose Value (mg/dL)', patient_6)
df_patient_6 = search_for_corresponding_data(6, patient_6, 'Glucose Value (mg/dL)')
df_patient_6

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
392,6,12/10/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),203.482639
400,6,12/18/19,1,89.93055556,5,24.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),143.451737
401,6,12/19/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),164.652778
402,6,12/20/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),149.756944
403,6,12/21/19,1,100,5,24.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),207.895833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,6,3/17/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,1,1,9,99,99,99,202.204861
485,6,3/18/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,1,1,8,99,99,99,157.569444
486,6,3/19/20,3,100.0,13,35.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,1,1,8,N/.A,NaN,NaN,174.704861
487,6,3/20/20,3,70.5,0,0.0,0,1,0,1,...,0,0,0,1,1,8,99,99,1,241.607930


In [11]:
# dexcom
patient_7 = read_and_combine_dexcom_without_parts(7)
patient_7 = fix_time(patient_7)
patient_7 = agreg_on_day_level('Glucose Value (mg/dL)', patient_7)
df_patient_7 = search_for_corresponding_data(7, patient_7, 'Glucose Value (mg/dL)')
df_patient_7

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
492,7,12/9/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
493,7,12/10/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
494,7,12/11/19,1,100,30,129.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
495,7,12/12/19,1,98.61111111,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
497,7,12/14/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,7,3/15/20,3,100,0,0.0,1,0,0,0,...,0,0,0,0,0,10,99,99,99,NaN
584,7,3/16/20,3,100,0,0.0,0,0,0,1,...,0,0,0,0,0,7,99,99,1,NaN
585,7,3/17/20,3,100,32,89.6,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,1,0,0,10,N/.A,NaN,NaN,NaN
586,7,3/18/20,3,100,0,0.0,1,0,0,0,...,0,0,0,0,0,10,99,99,99,NaN


In [12]:
# Dexcom Data PreProcessing
patient_8 = read_and_combine_dexcom_with_parts(8)
patient_8 = fix_time(patient_8)
patient_8 = agreg_on_day_level('Glucose Value (mg/dL)', patient_8)
df_patient_8 = search_for_corresponding_data(8, patient_8, 'Glucose Value (mg/dL)')
df_patient_8

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
588,8,12/8/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),254.145833
589,8,12/9/19,1,97.91666667,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),278.858156
590,8,12/10/19,1,90.625,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),271.808429
591,8,12/11/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),288.354167
592,8,12/12/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),297.413194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,8,3/25/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,1,1,7,99,99,99,242.177700
682,8,3/27/20,3,99.7,0,0.0,.,.,.,.,...,.,.,0,1,0,7,.,.,.,254.418118
683,8,3/28/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,0,1,4,99,99,99,243.156250
684,8,3/29/20,3,100.0,0,0.0,0,0,0,0,...,0,0,0,1,1,7,99,99,99,200.600694


In [13]:
# Dexcom Data PreProcessing
patient_10 = read_and_combine_dexcom_with_parts(10)
patient_10 = fix_time(patient_10)
patient_10 = agreg_on_day_level('Glucose Value (mg/dL)', patient_10)
df_patient_10 = search_for_corresponding_data(10, patient_10, 'Glucose Value (mg/dL)')
df_patient_10

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
788,10,1/5/20,1,91.31944444,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),251.125475
789,10,1/6/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),201.798611
790,10,1/7/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),171.197917
791,10,1/8/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),236.368056
792,10,1/9/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),199.048611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,10,4/8/20,3,100.0,20,76.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,7,N/.A,NaN,NaN,218.972222
878,10,4/9/20,3,100.0,20,76.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,7,N/.A,NaN,NaN,276.586806
879,10,4/10/20,3,100.0,32,121.6,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,1,7,N/.A,NaN,NaN,198.163194
880,10,4/11/20,3,100.0,31,117.8,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,8,N/.A,NaN,NaN,242.472222


In [14]:
# Librelink Data Preprocessing
patient_11 = read_and_combine_librelink_without_parts(11)
patient_11 = fix_time_librelink(patient_11)
patient_11 = agreg_on_day_level('Historic Glucose mg/dL', patient_11)
df_patient_11 = search_for_corresponding_data(11, patient_11, 'Historic Glucose mg/dL')
df_patient_11

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
882,11,4/18/20,1,98.95833333,40,320.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
883,11,4/19/20,1,89.58333333,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
884,11,4/20/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
885,11,4/21/20,1,95.83333333,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
886,11,4/22/20,1,87.5,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,11,7/22/20,3,96.875,30,294.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,1,1,8,N/.A,NaN,NaN,NaN
976,11,7/23/20,3,98.95833333,19,186.2,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,2,1,6,N/.A,NaN,NaN,NaN
977,11,7/24/20,3,93.75,0,0.0,0,0,0,0,...,0,0,0,1,0,8,99,99,99,NaN
978,11,7/25/20,3,95.83333333,15,157.5,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,1,1,7,N/.A,NaN,NaN,NaN


In [15]:
# Dexcom Data PreProcessing
patient_12 = read_and_combine_dexcom_with_parts(12)
patient_12 = fix_time(patient_12)
patient_12 = agreg_on_day_level('Glucose Value (mg/dL)', patient_12)
df_patient_12 = search_for_corresponding_data(12, patient_12, 'Glucose Value (mg/dL)')
df_patient_12

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
980,12,4/24/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),144.354167
981,12,4/25/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),113.597222
982,12,4/26/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),123.829861
983,12,4/27/20,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),124.684028
984,12,4/28/20,1,89.58333333,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),120.821705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,12,8/1/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,0,0,7,99,99,99,120.055556
1074,12,8/2/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,0,0,6,99,99,99,113.562500
1075,12,8/3/20,3,90.6,0,0.0,0,1,0,0,...,1,0,0,0,0,8,99,2,99,110.241379
1076,12,8/4/20,3,100.0,0,0.0,0,1,0,0,...,0,0,0,0,0,8,99,99,99,111.899306
